## Data log of entries, exits and P&L

In [2]:
import pandas as pd

# bt_df is the DataFrame returned by backtest()
contract_size = 1  # for equity

# Make sure bt_df is indexed by the `date` column:
df = bt_df.set_index("date")

# Extract entry & exit rows
entries = df[df.signal.isin(["BUY","SELL"])].copy()
exits   = df[df.signal == "EXIT"].copy()

trades = []
# Iterate over matching entry/exit pairs
for (entry_time, ent_row), (exit_time, ex_row) in zip(entries.iterrows(), exits.iterrows()):
    side        = ent_row.signal
    entry_price = ent_row.close
    exit_price  = ex_row.close
    qty         = int(ent_row.qty)
    notional    = entry_price * qty * contract_size
    pnl         = float(ex_row.pnl)
    duration    = exit_time - entry_time

    trades.append({
        "side":        side,
        "entry_time":  entry_time,   # now a Timestamp
        "exit_time":   exit_time,    # now a Timestamp
        "entry_price": entry_price,
        "exit_price":  exit_price,
        "qty":         qty,
        "notional":    notional,
        "pnl":         pnl,
        "duration":    duration,
    })

report = pd.DataFrame(trades)
print(report.to_string(index=False))
print(f"\nTOTAL PnL: ₹{report.pnl.sum():.2f}")

side                entry_time                 exit_time  entry_price  exit_price  qty  notional    pnl        duration
 BUY 2025-04-25 15:00:00+05:30 2025-04-28 13:15:00+05:30       1303.6      1368.9    7    9125.2  457.1 2 days 22:15:00
 BUY 2025-04-28 14:20:00+05:30 2025-05-05 09:40:00+05:30       1366.1      1438.4    7    9562.7  506.1 6 days 19:20:00
 BUY 2025-05-05 10:10:00+05:30 2025-05-06 11:15:00+05:30       1435.5      1412.0    6    8613.0 -141.0 1 days 01:05:00
SELL 2025-05-08 09:50:00+05:30 2025-05-12 13:40:00+05:30       1409.8      1431.0    7    9868.6 -148.4 4 days 03:50:00
 BUY 2025-05-13 09:20:00+05:30 2025-05-22 09:45:00+05:30       1431.0      1408.4    6    8586.0 -135.6 9 days 00:25:00
 BUY 2025-05-23 11:10:00+05:30 2025-06-02 09:15:00+05:30       1430.5      1401.1    6    8583.0 -176.4 9 days 22:05:00
SELL 2025-06-02 09:40:00+05:30 2025-06-03 10:20:00+05:30       1400.5      1422.5    7    9803.5 -154.0 1 days 00:40:00
 BUY 2025-06-05 11:20:00+05:30 2025-06-1

In [1]:
# ── Intraday scalping back-test (Reliance) ───────────────────────────
#
# The only change: build features **before** calling backtest().
# Everything else (parameters, model, etc.) stays the same.
# --------------------------------------------------------------------

# 1️⃣ Imports
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.features   import add_indicators               # ← already toggles
from algo.model      import load_or_train
from algo.backtester import backtest
import pandas as pd

# 2️⃣ Parameters  ─────────────────────────────────────────────────────
INTERVAL   = "3minute"
TRAIN_DAYS = 180
TEST_DAYS  = 20

CAPITAL    = 100_000
SL_PCT     = 0.0010
TP_PCT     = 0.0025
TRAIL_PCT  = 0.0025
HOLD_MAX   = 15

UPPER_PROB = 0.63
LOWER_PROB = 0.36

# 3️⃣ Fetch & prepare training data  ─────────────────────────────────
cfg        = load_config()
broker     = KiteWrapper(cfg)

hist_train = broker.history(days=TRAIN_DAYS, interval=INTERVAL)
print("Train interval:", pd.infer_freq(hist_train.index[:10]))
print(f"TRAIN: {len(hist_train):,} bars  |  "
      f"{hist_train.index.min()}  →  {hist_train.index.max()}")

df_train   = add_indicators(hist_train)          # ← compute all features
model      = load_or_train(df_train, retrain=True)

# 4️⃣ Fetch & prepare test slice  ────────────────────────────────────
hist_test  = broker.history(days=TEST_DAYS, interval=INTERVAL)
print(f"TEST : {len(hist_test):,} bars  |  "
      f"{hist_test.index.min()}  →  {hist_test.index.max()}")

df_test    = add_indicators(hist_test)           # ← compute all features

# 5️⃣ Run back-test  (logic now matches live loop 1-for-1)  ──────────
bt_df, metrics = backtest(
    df_test,                      # <-- pass **feature-enriched** DataFrame
    model=model,
    capital=CAPITAL,
    contract_size=1,
    sl_pct=SL_PCT,
    tp_pct=TP_PCT,
    trail_pct=TRAIL_PCT,
    hold_max=HOLD_MAX,
    upper=UPPER_PROB,
    lower=LOWER_PROB,
)

# 6️⃣ Review results  ────────────────────────────────────────────────
print("Back-test metrics:", metrics)


Train interval: 3min
TRAIN: 15,278 bars  |  2024-12-30 09:15:00+05:30  →  2025-06-26 10:36:00+05:30
Building X … done  (0.0s)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [1]:
# Intraday scalping back-test  (Reliance, 3-minute bars)

from algo.broker       import KiteWrapper
from algo.config       import load_config
from algo.features     import add_indicators
from algo.model        import load_or_train
from algo.backtester   import backtest
import pandas as pd

# ─── parameters ──────────────────────────────────────────────────────
INTERVAL     = "3minute"
TRAIN_DAYS   = 180
TEST_DAYS    = 20

CAPITAL      = 100_000
SL_PCT       = 0.0010
TP_PCT       = 0.0025
TRAIL_PCT    = 0.0015
HOLD_MAX     = 15

UPPER_PROB   = 0.63
LOWER_PROB   = 0.36

# ─── fetch & train ───────────────────────────────────────────────────
cfg        = load_config()
broker     = KiteWrapper(cfg)

hist_train = broker.history(days=TRAIN_DAYS, interval=INTERVAL)
print("Train interval:", pd.infer_freq(hist_train.index[:10]))
print(f"TRAIN: {len(hist_train):,} bars  |  "
      f"{hist_train.index.min()} → {hist_train.index.max()}")

model      = load_or_train(add_indicators(hist_train), retrain=False)

# ─── test slice & back-test ──────────────────────────────────────────
hist_test  = broker.history(days=TEST_DAYS, interval=INTERVAL)
print(f"TEST : {len(hist_test):,} bars  |  "
      f"{hist_test.index.min()} → {hist_test.index.max()}")

trades, metrics = backtest(
    hist_test,
    model=model,
    capital=CAPITAL,
    contract_size=1,
    sl_pct=SL_PCT,
    tp_pct=TP_PCT,
    trail_pct=TRAIL_PCT,
    hold_max=HOLD_MAX,
    upper=UPPER_PROB,
    lower=LOWER_PROB,
)

print("\nBack-test metrics:", metrics)


Train interval: 3min
TRAIN: 15,289 bars  |  2024-12-30 09:15:00+05:30 → 2025-06-26 11:09:00+05:30


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().